In [1]:
import cv2
import numpy as np
import os

def detect_lanes(frame, frame_filename, output_dir):

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)

    height, width = edges.shape
    vertices = np.array([[(0, height), (width // 2, height // 2), (width, height)]], dtype=np.int32)
    mask = np.zeros_like(edges)
    cv2.fillPoly(mask, vertices, 255)
    masked_edges = cv2.bitwise_and(edges, mask)

    lines = cv2.HoughLinesP(masked_edges, 1, np.pi / 180, 100, minLineLength=50, maxLineGap=10)

    # Draw lanes on the original frame
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(frame, (x1, y1), (x2, y2), (0, 0, 255), 2) 

    # Save the annotated frame
    output_path = os.path.join(output_dir, os.path.splitext(frame_filename)[0] + '_lanes.jpg')  
    cv2.imwrite(output_path, frame)

    return lines  


if __name__ == "__main__":
    frames_dir = "extracted_frames/"
    output_dir = "lanes_visualized/"  
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(frames_dir):
        if filename.endswith(".jpg"):
            frame_path = os.path.join(frames_dir, filename)
            frame = cv2.imread(frame_path)

            
            detect_lanes(frame, filename, output_dir)

